In [1]:
#def init():
    #CHANGE THIS TO YOUR TRAIN FOLDER
image_folder = 'C:/Study/Semester2/Machine Learning/ML_images/Training' 
#image_folder = 'C:/Users/danie/Desktop/Training_ML'

TOO_SMALL = 200000
SMALL = 500000
MEDIUM = 1200000
LARGE = 2100000
    
X_train = []
Y_train = []
counter = 1
    
img_height = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
img_width = 1888 #Usually 128. this might change depending on Annaya & Danielle's input 
epochs = 10 #Start with 1, and increase to 10, 100, 500, 1000 and 3000. 11 is the recommended number of runs through the training dataset. We will probably have to tune this. 
#total_training = total number of training data len()
batch_size = 1 #test different ones from 1 to 2, 5, 10, etc
total_images = 1

In [2]:
#pip install opencv-python

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage
import cv2
import glob
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, UpSampling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage import data, io,color
from skimage.io import imread_collection #loads a collection of images
from skimage.io import imread, imshow
from pathlib import Path

from sklearn.decomposition import PCA

import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
#Define sizes of images based on their resolution
#Image shape and resolution

#TODO make the IMG sizes make sense


def shape(img):
    size = 0
    resolution = img.shape[0] * img.shape[1]
    if resolution <= TOO_SMALL:
        size = 0
    elif resolution <= SMALL:
        size = 1
    elif resolution <= MEDIUM:
        size = 2
    elif resolution <= LARGE:
        size = 3
    else:
        size = 6
    return size

In [5]:
def create_dataframe(image_folder):
    images = Path(image_folder).glob('*.jpg')

    list_of_files = []
    for image in images:
        list_of_files.append(str(image))

    df = pd. DataFrame(list_of_files, columns = ["image_path"])
    df_length = len(df.index)
    print("processing", df_length, "photos for training")
    
    height= []
    width = []
    resolution = []
    
    for image_path in df['image_path']:
        img = cv2.imread(str(image_path))
        height.append(img.shape[0])
        width.append(img.shape[1])
        
    df["height"] = height
    df["width"] = width
    df["resolution"] = df["height"] * df["width"]
    
    df = df[df.resolution < LARGE]
    
    ll = df_length - len(df.index)
    print("dropped ", ll, "photos because they were too large to process")
    
    df = df[df.resolution > TOO_SMALL]
    
    ss = df_length - len(df.index) - ll
    print("dropped ", ss, "photos because they were too small to process")
    df.head()
    
    return df

In [6]:
#PCA rgb then pass to prep_img as an rgb image

def pca_rgb(imgBGR, size): #imgRGB, size
    b,g,r = cv2.split(imgBGR)
    list_channels = [b,g,r]
    
    #check size first
    if size ==1:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 100)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)    

    elif size == 2:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 300)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
    else:
        inverted_img = []
        for c in list_channels:            
            pca = PCA(n_components = 500)
            transformed = pca.fit_transform(c/255) #r_scaled
            inverted = pca.inverse_transform(transformed)
            inverted_img.append(inverted)
        
        tuple(inverted_img)
        RGB_compressed = cv2.merge(inverted_img)
        
    RGB_compressed = np.float32(RGB_compressed)
        
    return RGB_compressed

In [7]:
# image padding

def rgb_pad(image):
    
    height = image.shape[0]
    width = image.shape[1]

    WHITE = [255,255,255] #[255,255,255]  1, 1, 1
    MAX = 1888
    
    if height % 2 == 0:
        top = int((MAX - height)/ 2)
        bottom = int((MAX - height)/ 2)
    else: 
        top = int((MAX - (height-1))/ 2)
        bottom = int(((MAX - (height-1))/ 2)-1)

    if width % 2 == 0:
        left = int((MAX - width)/ 2)
        right = int((MAX - width)/ 2)
    else: 
        left = int((MAX - (width-1))/ 2)
        right = int(((MAX - (width-1))/2)-1)
    
    image = cv2.copyMakeBorder(image, top, bottom, left, right,
    cv2.BORDER_CONSTANT,value=WHITE)

    return image

In [8]:
#Debugged function
def prep_img(img_path):
    imgRGB = img_to_array(load_img(img_path))
    imgRGB = np.array(imgRGB, dtype=float)

    
    #check the image category by 'shape' function
    image_size_category = shape(imgRGB) 
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
 
    RGB_compressed = pca_rgb(imgRGB, image_size_category) 
    print('RGB_compressed:', np.min(RGB_compressed), np.max(RGB_compressed))
    #plt.figure(figsize=(10, 10))
    #plt.imshow(RGB_compressed)
    
    print("image is padded")
    padded_image = rgb_pad(RGB_compressed)
  
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image) # show after rgb compressed after padded
    
    #image_pca = np.float32((1.0/255)*image_pca) # #The rgb_compressed come out in RGB scaled data already
    print("image converted to LAB")
    padded_LAB = cv2.cvtColor(padded_image, cv2.COLOR_RGB2Lab) #convert padded RGB compressed to LAB
    print('Padded LAB:', np.min(padded_LAB), np.max(padded_LAB))
    
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image, cmap = plt.cm.gray) #show after convert padded rgb compressed to RGB with the function
  
    #separate L and AB color channels
    Y = padded_LAB[:, :,1:]
    Y = Y / 128 #comment out when show predicted
    print('A normalized:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    print('B normalized:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    
    X = padded_LAB[:, :, 0] 
    print('X:', np.min(X), np.max(X))
    #plt.figure(figsize=(10, 10)) # show converted RGB to L channel 
    #plt.imshow(X)
    
    #this is in sample code and idk why
    
    X = X.reshape(1, 1888, 1888, 1) #comment out when show predicted
    Y = Y.reshape(1, 1888, 1888, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X 

In [9]:
def prep_img_no_pca(img_path):
    imgRGB = img_to_array(load_img(img_path)) #cv2.imread(img_path)
    print('imgRGB:', np.min(imgRGB), np.max(imgRGB))
    imgRGB = np.array(imgRGB, dtype=float)

    
    #check the image category by 'shape' function
    image_size_category = shape(imgRGB) #imgBGR
    #print("image size determined!")

    if image_size_category <1:
        raise Exception("Your image is too small to be used")
    elif image_size_category >4:
        raise Exception("Your image is too big to be used")

    #print("image PCA started!")
    #image_pca = pca_rgb(imgRGB, image_size_category) 
    #RGB_compressed = pca_rgb(imgRGB, image_size_category) #imgBGR, image_size_category
    #RGB_compressed = np.int32(RGB_compressed*255)
    #print('RGB_compressed:', np.min(RGB_compressed), np.max(RGB_compressed))
    #plt.figure(figsize=(10, 10))
    #plt.imshow(RGB_compressed)
    
    print("image is padded")
    #padded_image = rgb_pad(RGB_compressed)
    padded_image = rgb_pad(imgRGB)
    
    print('padded_image:', np.min(padded_image), np.max(padded_image))
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image) # show after rgb compressed after padded
    
    print("image converted to LAB")
   
    X_LAB = color.rgb2lab(1.0/255*padded_image)[:,:,0] #1.0/255*padded_image
    print('X_LAB:', np.min(X_LAB), np.max(X_LAB))
    Y_LAB = color.rgb2lab(1.0/255*padded_image)[:,:,1:] #1.0/255*padded_image
    Y_LAB = Y_LAB / 128
    print('Y_LAB:', np.min(Y_LAB), np.max(Y_LAB))
    
    #plt.figure(figsize=(10, 10))
    #plt.imshow(padded_image, cmap = plt.cm.gray) #show after convert padded rgb compressed to RGB with the function
    #imgLAB = np.clip(imgLAB, 0, 1)

    
    #separate L and AB color channels
    #Y = padded_LAB[:, :,1:]
    #Y = Y / 128 #comment out when show predicted
    #print('A:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    #print('B:', np.min(Y[:,:,0]), np.max(Y[:,:,0]))
    
    #X = padded_LAB[:, :, 0] 
    #print('X:', np.min(X), np.max(X))
    #plt.figure(figsize=(10, 10)) # show converted RGB to L channel 
    #plt.imshow(X)
    
    #this is in sample code and idk why
    
    X = X_LAB.reshape(1, 1888, 1888, 1) #comment out when show predicted
    Y = Y_LAB.reshape(1, 1888, 1888, 2) #comment out when show predicted
    
    print("image processed")

    return Y, X 

In [10]:
def prep_dataframe(df):
    for index, row in df.iterrows():
        Y, X = prep_img(row["image_path"])
        Y_train.append(Y)
        X_train.append(X)

In [11]:
def prep_dataframe_no_pca(df):
    for index, row in df.iterrows():
        Y, X = prep_img_no_pca(row["image_path"])
        Y_train.append(Y)
        X_train.append(X)

# Use the code below to process ALL photos and feed them to the model!!!

In [12]:
df = create_dataframe(image_folder)
prep_dataframe_no_pca(df)



processing 151 photos for training
dropped  9 photos because they were too large to process
dropped  5 photos because they were too small to process
imgRGB: 0.0 253.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.3921558348327565 100.0
Y_LAB: -0.32814414307843537 0.35772671860956773
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.38468816384518295 0.6362840363931217
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.38561733685524413 0.33376367721236444
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.21970575246488613 0.5544613180150035
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.05830875157287707 100.0
Y_LAB: -0.24268470903432968 0.29287547327793306
image processed
imgRGB: 0.0 

X_LAB: 0.039573654069084085 100.0
Y_LAB: -0.4703806545152792 0.35416838413731766
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.2236486071056207 0.3162899246299724
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.31117034072506095 0.38872444517933
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.45506613573345406 0.5845765811449297
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.09893413517270844 100.0
Y_LAB: -0.18685212616131266 0.3208976162686566
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.7035639052764466 100.0
Y_LAB: -0.3900111398930429 0.3580159788301473
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image convert

X_LAB: 0.0 100.0
Y_LAB: -0.32244638820824856 0.3780117902825285
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.3153021546480502 0.3440383217198535
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.07914730813816462 100.0
Y_LAB: -0.38889016948320465 0.5544461759482899
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.4660402976357113 0.5075847982011173
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.5788884331186912 0.5878954640099067
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LAB: -0.5266158284499761 0.44293480185227924
image processed
imgRGB: 0.0 255.0
image is padded
padded_image: 0.0 255.0
image converted to LAB
X_LAB: 0.0 100.0
Y_LA

In [15]:
def mode(): 
    model2 = Sequential([
        #encoder
        Conv2D(128, 2, strides = 2, padding = "same", activation = "relu", input_shape= (1888, 1888, 1)), #testing shape changes
        Conv2D(256, 2, strides = 2, padding = "same", activation = "relu"), 
        Conv2D(512, 2, strides = 2, padding = "same", activation = "relu"), #filter = 256
        
        #decoder
        Conv2D(256, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2),
        Conv2D(128, 2, padding = "same", activation = "relu"), 
        UpSampling2D(2), 
        Conv2D(2, 2, padding = "same", activation = "tanh"),
        UpSampling2D(2),
    ])
    model2.compile(optimizer ="adam", loss = "mse", metrics = ["accuracy"])
    return model2

In [16]:
model2 = mode()
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 944, 944, 128)     640       
                                                                 
 conv2d_7 (Conv2D)           (None, 472, 472, 256)     131328    
                                                                 
 conv2d_8 (Conv2D)           (None, 236, 236, 512)     524800    
                                                                 
 conv2d_9 (Conv2D)           (None, 236, 236, 256)     524544    
                                                                 
 up_sampling2d_3 (UpSampling  (None, 472, 472, 256)    0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 472, 472, 128)     131200    
                                                      

In [17]:
len(X_train)

137

In [18]:
len(Y_train)

137

In [22]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

In [23]:
epochs = 1
history = model2.fit(train_data, batch_size = batch_size, epochs = epochs, verbose = 2)  # add validation_split = 0.2

137/137 - 509s - loss: 1.0017 - accuracy: 0.1198 - 509s/epoch - 4s/step


In [24]:
filename = 'my_model_tf2_E1'
model2.save(filename)

INFO:tensorflow:Assets written to: my_model_tf2_E1\assets


## Load model and test in Test notebook